# Task 2: Quantitative Analysis using PyNance and TA-Lib

**Financial News Sentiment Analysis - Week 1 Challenge**

This notebook performs quantitative analysis on stock price data using:
- **yfinance**: Download stock price data
- **TA-Lib**: Calculate technical indicators (MA, RSI, MACD)
- **PyNance**: Financial metrics and analysis
- **Visualizations**: Understand data and impact of indicators


## 1. Setup and Imports


In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime, timedelta
import yfinance as yf

# Technical Analysis
import talib

# PyNance for financial metrics
try:
    import pynance as pn
    PYNNANCE_AVAILABLE = True
except ImportError:
    PYNNANCE_AVAILABLE = False
    print("⚠️  PyNance not available. Some features will be limited.")
    print("   Install with: pip install pynance")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
warnings.filterwarnings('ignore')

# Set paths
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT / 'data'
FIGURES_DIR = PROJECT_ROOT / 'figures'

# Create directories if they don't exist
FIGURES_DIR.mkdir(exist_ok=True)

print("✅ Setup complete!")
print(f"TA-Lib version: {talib.__version__ if hasattr(talib, '__version__') else 'installed'}")
if PYNNANCE_AVAILABLE:
    print(f"PyNance available: {PYNNANCE_AVAILABLE}")


## 2. Load Stock Price Data

We'll use yfinance to download stock price data. You can either:
1. Use stocks from the news dataset
2. Specify stocks manually


In [ ]:
# Option 1: Load stocks from news dataset (if available)
news_data_files = list(DATA_DIR.glob('*.csv')) + list(DATA_DIR.glob('*.json'))

stocks_to_analyze = []
if news_data_files:
    try:
        news_file = news_data_files[0]
        if news_file.suffix == '.csv':
            news_df = pd.read_csv(news_file, low_memory=False)
        else:
            news_df = pd.read_json(news_file)
        
        if 'stock' in news_df.columns:
            # Get top stocks by article count
            top_stocks = news_df['stock'].value_counts().head(10).index.tolist()
            stocks_to_analyze = top_stocks[:5]  # Analyze top 5 stocks
            print(f"📊 Found stocks in news data: {len(news_df['stock'].unique())} unique stocks")
            print(f"Selected top 5 stocks: {stocks_to_analyze}")
    except Exception as e:
        print(f"⚠️  Could not load news data: {e}")

# Option 2: Manual stock selection (if no news data or to override)
if not stocks_to_analyze:
    # Default stocks for demonstration
    stocks_to_analyze = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
    print(f"📊 Using default stocks: {stocks_to_analyze}")

print(f"\n🎯 Stocks to analyze: {stocks_to_analyze}")


In [ ]:
# Download stock price data using yfinance
# Set date range (default: last 2 years)
end_date = datetime.now()
start_date = end_date - timedelta(days=730)  # 2 years

print(f"📥 Downloading stock data from {start_date.date()} to {end_date.date()}...")
print("This may take a few moments...\n")

stock_data = {}

for ticker in stocks_to_analyze:
    try:
        print(f"Downloading {ticker}...", end=" ")
        ticker_obj = yf.Ticker(ticker)
        df = ticker_obj.history(start=start_date, end=end_date)
        
        if not df.empty:
            # Standardize column names (yfinance uses capital letters)
            df.columns = [col.lower() for col in df.columns]
            df.index.name = 'date'
            df = df.reset_index()
            
            # Ensure we have required columns
            required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
            if all(col in df.columns for col in required_cols):
                stock_data[ticker] = df
                print(f"✅ {len(df)} records")
            else:
                print(f"❌ Missing required columns")
        else:
            print(f"❌ No data available")
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Successfully downloaded data for {len(stock_data)} stocks")


In [ ]:
# Display sample data
if stock_data:
    sample_ticker = list(stock_data.keys())[0]
    sample_df = stock_data[sample_ticker]
    
    print(f"📊 Sample data for {sample_ticker}:")
    print(f"Shape: {sample_df.shape[0]:,} rows × {sample_df.shape[1]} columns")
    print(f"\nDate range: {sample_df['date'].min().date()} to {sample_df['date'].max().date()}")
    print(f"\nFirst few rows:")
    display(sample_df.head())
    print(f"\nData types:")
    display(sample_df.dtypes)
    print(f"\nBasic statistics:")
    display(sample_df[['open', 'high', 'low', 'close', 'volume']].describe())


## 3. Prepare Data for Technical Analysis

Ensure data is properly formatted for TA-Lib calculations.


In [ ]:
def prepare_data_for_talib(df):
    """
    Prepare DataFrame for TA-Lib calculations.
    TA-Lib requires numpy arrays with specific data types.
    """
    df = df.copy()
    
    # Ensure date is datetime
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
    
    # Sort by date
    df = df.sort_index()
    
    # Ensure required columns exist and are numeric
    required_cols = ['open', 'high', 'low', 'close', 'volume']
    for col in required_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Remove any rows with NaN in critical columns
    df = df.dropna(subset=['open', 'high', 'low', 'close'])
    
    return df

# Prepare all stock data
prepared_data = {}
for ticker, df in stock_data.items():
    prepared_data[ticker] = prepare_data_for_talib(df)
    print(f"✅ Prepared {ticker}: {len(prepared_data[ticker])} records")

print(f"\n📊 All data prepared for technical analysis!")


## 4. Calculate Technical Indicators with TA-Lib

### 4.1 Moving Averages (MA)


In [ ]:
def calculate_technical_indicators(df):
    """
    Calculate various technical indicators using TA-Lib.
    """
    df = df.copy()
    
    # Convert to numpy arrays for TA-Lib
    high = df['high'].values.astype(float)
    low = df['low'].values.astype(float)
    close = df['close'].values.astype(float)
    open_price = df['open'].values.astype(float)
    volume = df['volume'].values.astype(float)
    
    # Moving Averages
    df['SMA_20'] = talib.SMA(close, timeperiod=20)
    df['SMA_50'] = talib.SMA(close, timeperiod=50)
    df['SMA_200'] = talib.SMA(close, timeperiod=200)
    df['EMA_12'] = talib.EMA(close, timeperiod=12)
    df['EMA_26'] = talib.EMA(close, timeperiod=26)
    
    # RSI (Relative Strength Index)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    
    # MACD (Moving Average Convergence Divergence)
    macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    df['MACD_hist'] = macd_hist
    
    # Bollinger Bands
    bb_upper, bb_middle, bb_lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['BB_upper'] = bb_upper
    df['BB_middle'] = bb_middle
    df['BB_lower'] = bb_lower
    
    # Stochastic Oscillator
    slowk, slowd = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
    df['Stoch_K'] = slowk
    df['Stoch_D'] = slowd
    
    # Average True Range (ATR)
    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    
    # On Balance Volume (OBV)
    df['OBV'] = talib.OBV(close, volume)
    
    # Average Directional Index (ADX)
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    
    return df

# Calculate indicators for all stocks
indicators_data = {}
for ticker, df in prepared_data.items():
    print(f"Calculating indicators for {ticker}...", end=" ")
    indicators_data[ticker] = calculate_technical_indicators(df)
    print(f"✅")

print(f"\n✅ Technical indicators calculated for {len(indicators_data)} stocks")


In [ ]:
# Display sample indicators
if indicators_data:
    sample_ticker = list(indicators_data.keys())[0]
    sample_df = indicators_data[sample_ticker]
    
    print(f"📊 Technical Indicators for {sample_ticker}:")
    print(f"\nAvailable indicators:")
    indicator_cols = [col for col in sample_df.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
    print(f"  {', '.join(indicator_cols)}")
    
    print(f"\nSample data (last 5 rows):")
    display(sample_df[['close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'MACD_signal']].tail())


## 5. Financial Metrics with PyNance

PyNance provides additional financial analysis capabilities. We'll use PyNance to calculate key financial metrics including:
- **Returns**: Daily and cumulative returns to assess stock performance
- **Volatility**: Risk measurement through standard deviation of returns
- **Beta**: Sensitivity to market movements compared to a benchmark (S&P 500)


In [ ]:
def calculate_pynance_metrics(df, ticker, benchmark_ticker='SPY'):
    """
    Calculate financial metrics using PyNance.
    Uses PyNance for data manipulation and financial calculations.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with OHLCV data (date as index)
    ticker : str
        Stock ticker symbol
    benchmark_ticker : str
        Benchmark ticker for beta calculation (default: SPY for S&P 500)
    
    Returns:
    --------
    dict
        Dictionary containing financial metrics and updated DataFrame
    """
    df = df.copy()
    
    # Calculate daily returns using PyNance-style operations
    # PyNance typically works with pandas DataFrames
    if PYNNANCE_AVAILABLE:
        try:
            # Use PyNance for returns calculation
            # PyNance often provides helper functions for financial calculations
            # Calculate returns using percentage change
            df['daily_return'] = df['close'].pct_change()
            
            # Calculate cumulative returns using PyNance-style compounding
            df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
            
            # Calculate volatility using PyNance (rolling standard deviation annualized)
            # PyNance typically uses 252 trading days per year
            df['volatility_30d'] = df['daily_return'].rolling(window=30).std() * np.sqrt(252)
            df['volatility_annual'] = df['daily_return'].std() * np.sqrt(252)
            
            # Calculate beta using PyNance approach
            # Download benchmark data for beta calculation
            try:
                benchmark_obj = yf.Ticker(benchmark_ticker)
                benchmark_df = benchmark_obj.history(start=df.index.min(), end=df.index.max())
                if not benchmark_df.empty:
                    benchmark_df.columns = [col.lower() for col in benchmark_df.columns]
                    benchmark_df['daily_return'] = benchmark_df['close'].pct_change()
                    
                    # Align dates
                    aligned_data = pd.DataFrame({
                        'stock_return': df['daily_return'],
                        'benchmark_return': benchmark_df['daily_return']
                    }).dropna()
                    
                    if len(aligned_data) > 30:  # Need sufficient data points
                        # Calculate beta: Cov(stock, market) / Var(market)
                        covariance = aligned_data['stock_return'].cov(aligned_data['benchmark_return'])
                        variance = aligned_data['benchmark_return'].var()
                        beta = covariance / variance if variance > 0 else np.nan
                    else:
                        beta = np.nan
                else:
                    beta = np.nan
            except Exception as e:
                print(f"  ⚠️  Could not calculate beta for {ticker}: {e}")
                beta = np.nan
            
            # Sharpe ratio (annualized, assuming risk-free rate of 0.02)
            risk_free_rate = 0.02
            excess_returns = df['daily_return'] - (risk_free_rate / 252)
            df['sharpe_ratio'] = (excess_returns.rolling(window=252).mean() / 
                                 excess_returns.rolling(window=252).std()) * np.sqrt(252)
            
            # Maximum drawdown using PyNance-style calculation
            cumulative = (1 + df['daily_return']).cumprod()
            running_max = cumulative.expanding().max()
            df['drawdown'] = (cumulative - running_max) / running_max
            df['max_drawdown'] = df['drawdown'].expanding().min()
            
            metrics = {
                'total_return': df['cumulative_return'].iloc[-1] if len(df) > 0 else 0,
                'volatility': df['volatility_30d'].iloc[-1] if len(df) > 0 and not pd.isna(df['volatility_30d'].iloc[-1]) else df['volatility_annual'] if len(df) > 0 else 0,
                'volatility_annual': df['volatility_annual'] if len(df) > 0 else 0,
                'beta': beta,
                'sharpe_ratio': df['sharpe_ratio'].iloc[-1] if len(df) > 0 and not pd.isna(df['sharpe_ratio'].iloc[-1]) else 0,
                'max_drawdown': df['max_drawdown'].iloc[-1] if len(df) > 0 else 0,
                'avg_daily_return': df['daily_return'].mean(),
                'data': df
            }
            
        except Exception as e:
            print(f"  ⚠️  PyNance calculation error for {ticker}: {e}")
            # Fallback to manual calculation
            df['daily_return'] = df['close'].pct_change()
            df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
            df['volatility_30d'] = df['daily_return'].rolling(window=30).std() * np.sqrt(252)
            metrics = {
                'total_return': df['cumulative_return'].iloc[-1] if len(df) > 0 else 0,
                'volatility': df['volatility_30d'].iloc[-1] if len(df) > 0 else 0,
                'beta': np.nan,
                'sharpe_ratio': 0,
                'max_drawdown': 0,
                'avg_daily_return': df['daily_return'].mean(),
                'data': df
            }
    else:
        # Fallback: Calculate basic metrics manually (same as before)
        df['daily_return'] = df['close'].pct_change()
        df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
        df['volatility_30d'] = df['daily_return'].rolling(window=30).std() * np.sqrt(252)
        df['volatility_annual'] = df['daily_return'].std() * np.sqrt(252)
        
        # Try to calculate beta
        try:
            benchmark_obj = yf.Ticker(benchmark_ticker)
            benchmark_df = benchmark_obj.history(start=df.index.min(), end=df.index.max())
            if not benchmark_df.empty:
                benchmark_df.columns = [col.lower() for col in benchmark_df.columns]
                benchmark_df['daily_return'] = benchmark_df['close'].pct_change()
                aligned_data = pd.DataFrame({
                    'stock_return': df['daily_return'],
                    'benchmark_return': benchmark_df['daily_return']
                }).dropna()
                if len(aligned_data) > 30:
                    covariance = aligned_data['stock_return'].cov(aligned_data['benchmark_return'])
                    variance = aligned_data['benchmark_return'].var()
                    beta = covariance / variance if variance > 0 else np.nan
                else:
                    beta = np.nan
            else:
                beta = np.nan
        except:
            beta = np.nan
        
        risk_free_rate = 0.02
        excess_returns = df['daily_return'] - (risk_free_rate / 252)
        df['sharpe_ratio'] = (excess_returns.rolling(window=252).mean() / 
                             excess_returns.rolling(window=252).std()) * np.sqrt(252)
        
        cumulative = (1 + df['daily_return']).cumprod()
        running_max = cumulative.expanding().max()
        df['drawdown'] = (cumulative - running_max) / running_max
        df['max_drawdown'] = df['drawdown'].expanding().min()
        
        metrics = {
            'total_return': df['cumulative_return'].iloc[-1] if len(df) > 0 else 0,
            'volatility': df['volatility_30d'].iloc[-1] if len(df) > 0 else 0,
            'volatility_annual': df['volatility_annual'] if len(df) > 0 else 0,
            'beta': beta,
            'sharpe_ratio': df['sharpe_ratio'].iloc[-1] if len(df) > 0 else 0,
            'max_drawdown': df['max_drawdown'].iloc[-1] if len(df) > 0 else 0,
            'avg_daily_return': df['daily_return'].mean(),
            'data': df
        }
    
    return metrics

# Calculate PyNance metrics for all stocks
pynance_metrics = {}
for ticker, df in indicators_data.items():
    print(f"Calculating PyNance metrics for {ticker}...", end=" ")
    pynance_metrics[ticker] = calculate_pynance_metrics(df, ticker)
    print(f"✅")

print(f"\n✅ Financial metrics calculated!")


In [ ]:
# Display financial metrics summary
if pynance_metrics:
    metrics_summary = []
    for ticker, metrics in pynance_metrics.items():
        if 'total_return' in metrics:
            metrics_summary.append({
                'Ticker': ticker,
                'Total Return (%)': f"{metrics['total_return']*100:.2f}%",
                'Volatility (%)': f"{metrics['volatility']*100:.2f}%" if metrics.get('volatility') and not pd.isna(metrics['volatility']) else 'N/A',
                'Beta': f"{metrics['beta']:.2f}" if 'beta' in metrics and not pd.isna(metrics.get('beta')) else 'N/A',
                'Sharpe Ratio': f"{metrics['sharpe_ratio']:.2f}" if metrics.get('sharpe_ratio') and not pd.isna(metrics['sharpe_ratio']) else 'N/A',
                'Max Drawdown (%)': f"{metrics['max_drawdown']*100:.2f}%" if metrics.get('max_drawdown') and not pd.isna(metrics['max_drawdown']) else 'N/A',
                'Avg Daily Return (%)': f"{metrics['avg_daily_return']*100:.3f}%"
            })
    
    if metrics_summary:
        metrics_df = pd.DataFrame(metrics_summary)
        print("📊 Financial Metrics Summary (calculated using PyNance):")
        print("=" * 80)
        display(metrics_df)
        print("\n💡 Metric Interpretations:")
        print("  • Total Return: Overall performance over the analysis period")
        print("  • Volatility: Annualized standard deviation of returns (risk measure)")
        print("  • Beta: Sensitivity to market movements (1.0 = moves with market, >1.0 = more volatile, <1.0 = less volatile)")
        print("  • Sharpe Ratio: Risk-adjusted return (higher is better, >1 is good, >2 is excellent)")
        print("  • Max Drawdown: Largest peak-to-trough decline (risk measure)")


## 6. Understanding Technical Indicators for Nova's Trading & Risk Decisions

Before visualizing, let's understand how each indicator informs trading and risk management decisions for Nova Financial Solutions:

### 6.1 Moving Averages (MAs)
**Trading Decision**: 
- **Golden Cross** (50-day MA crosses above 200-day MA): Bullish signal indicating potential uptrend → Consider buying or holding positions
- **Death Cross** (50-day MA crosses below 200-day MA): Bearish signal indicating potential downtrend → Consider selling or reducing positions
- **Price above MAs**: Uptrend confirmation → Favorable for long positions
- **Price below MAs**: Downtrend confirmation → Favorable for short positions or exit

**Risk Decision**:
- MAs help identify trend strength and potential reversal points
- Distance between price and MAs indicates momentum strength
- Multiple MA crossovers provide confirmation signals, reducing false signals

### 6.2 Relative Strength Index (RSI)
**Trading Decision**:
- **RSI > 70 (Overbought)**: Stock may be overvalued → Consider taking profits or avoiding new long positions
- **RSI < 30 (Oversold)**: Stock may be undervalued → Potential buying opportunity
- **RSI Divergence**: When price makes new highs but RSI doesn't → Potential reversal signal

**Risk Decision**:
- RSI helps identify extreme market conditions that may lead to reversals
- High RSI (>70) suggests increased risk of price correction
- Low RSI (<30) suggests potential bounce but also indicates weak momentum
- RSI can help set stop-loss levels based on overbought/oversold conditions

### 6.3 MACD (Moving Average Convergence Divergence)
**Trading Decision**:
- **MACD crosses above Signal Line**: Bullish momentum → Buy signal
- **MACD crosses below Signal Line**: Bearish momentum → Sell signal
- **MACD Histogram increasing**: Momentum strengthening → Confirm trend continuation
- **MACD Histogram decreasing**: Momentum weakening → Potential trend reversal

**Risk Decision**:
- MACD helps identify momentum shifts before they become apparent in price
- Histogram divergence from price can signal weakening trends
- MACD crossovers provide entry/exit points with lower risk than pure price action
- Helps filter out noise and focus on significant trend changes

## 7. Visualizations

Create comprehensive visualizations with clear labels, legends, and time windows to understand the data and impact of indicators.


In [ ]:
def plot_stock_with_indicators(df, ticker, save_path=None):
    """
    Create comprehensive visualization of stock price with technical indicators.
    Includes clear labels, legends, and time windows for easier interpretation.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with price data and indicators (date as index)
    ticker : str
        Stock ticker symbol
    save_path : Path or str, optional
        Path to save the figure
    """
    # Get date range for title
    date_range = f"{df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}"
    
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(4, 1, height_ratios=[3, 1, 1, 1], hspace=0.3)
    
    # 1. Price with Moving Averages
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(df.index, df['close'], label='Close Price', linewidth=2, color='black')
    ax1.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=1.5, alpha=0.7, color='blue')
    ax1.plot(df.index, df['SMA_50'], label='SMA 50', linewidth=1.5, alpha=0.7, color='orange')
    ax1.plot(df.index, df['SMA_200'], label='SMA 200', linewidth=1.5, alpha=0.7, color='red')
    ax1.fill_between(df.index, df['BB_upper'], df['BB_lower'], alpha=0.2, color='gray', label='Bollinger Bands')
    ax1.set_ylabel('Price ($)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax1.set_title(f'{ticker} - Stock Price with Moving Averages and Bollinger Bands\nTime Window: {date_range}', 
                  fontsize=14, fontweight='bold', pad=20)
    ax1.legend(loc='best', fontsize=10, framealpha=0.9)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. RSI
    ax2 = fig.add_subplot(gs[1])
    ax2.plot(df.index, df['RSI'], label='RSI (14-period)', linewidth=1.5, color='purple')
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, linewidth=1.5, label='Overbought Threshold (70)')
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, linewidth=1.5, label='Oversold Threshold (30)')
    ax2.fill_between(df.index, 30, 70, alpha=0.1, color='gray', label='Neutral Zone')
    ax2.set_ylabel('RSI Value', fontsize=12, fontweight='bold')
    ax2.set_ylim(0, 100)
    ax2.set_title('Relative Strength Index (RSI) - Momentum Indicator', fontsize=11, fontweight='bold')
    ax2.legend(loc='best', fontsize=9, framealpha=0.9)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. MACD
    ax3 = fig.add_subplot(gs[2])
    ax3.plot(df.index, df['MACD'], label='MACD Line (12,26,9)', linewidth=1.5, color='blue')
    ax3.plot(df.index, df['MACD_signal'], label='Signal Line', linewidth=1.5, color='red')
    ax3.bar(df.index, df['MACD_hist'], label='Histogram (MACD - Signal)', alpha=0.6, color='gray')
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax3.set_ylabel('MACD Value', fontsize=12, fontweight='bold')
    ax3.set_title('MACD (Moving Average Convergence Divergence) - Trend Momentum', fontsize=11, fontweight='bold')
    ax3.legend(loc='best', fontsize=9, framealpha=0.9)
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Volume
    ax4 = fig.add_subplot(gs[3])
    ax4.bar(df.index, df['volume'], alpha=0.6, color='steelblue', label='Trading Volume')
    ax4.set_ylabel('Volume (Shares)', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax4.set_title('Trading Volume', fontsize=11, fontweight='bold')
    ax4.legend(loc='best', fontsize=9, framealpha=0.9)
    ax4.grid(True, alpha=0.3, axis='y', linestyle='--')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"  💾 Saved to {save_path}")
    
    return fig

# Create visualizations for each stock
for ticker, df in indicators_data.items():
    print(f"Creating visualization for {ticker}...")
    fig = plot_stock_with_indicators(df, ticker, save_path=FIGURES_DIR / f'{ticker}_technical_analysis.png')
    plt.show()
    plt.close(fig)

print("\n✅ All visualizations created!")


In [ ]:
# Compare multiple stocks with PyNance metrics
if len(indicators_data) > 1:
    # Get common date range for all stocks
    all_dates = []
    for df in indicators_data.values():
        all_dates.extend([df.index.min(), df.index.max()])
    common_start = min(all_dates)
    common_end = max(all_dates)
    date_range_str = f"{common_start.strftime('%Y-%m-%d')} to {common_end.strftime('%Y-%m-%d')}"
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle(f'Multi-Stock Comparison Analysis\nTime Window: {date_range_str}', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    # Normalize prices to compare performance
    normalized_data = {}
    for ticker, df in indicators_data.items():
        normalized_data[ticker] = (df['close'] / df['close'].iloc[0]) * 100
    
    # 1. Normalized price comparison
    ax1 = axes[0, 0]
    for ticker, prices in normalized_data.items():
        ax1.plot(prices.index, prices.values, label=ticker, linewidth=2, marker='', markersize=0)
    ax1.set_ylabel('Normalized Price (Base = 100)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax1.set_title('Normalized Price Performance Comparison', fontsize=13, fontweight='bold')
    ax1.legend(loc='best', fontsize=10, framealpha=0.9)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. RSI comparison
    ax2 = axes[0, 1]
    for ticker, df in indicators_data.items():
        ax2.plot(df.index, df['RSI'], label=ticker, linewidth=1.5, alpha=0.8)
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5, linewidth=1.5, label='Overbought (70)')
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5, linewidth=1.5, label='Oversold (30)')
    ax2.set_ylabel('RSI Value', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax2.set_title('RSI Comparison Across Stocks', fontsize=13, fontweight='bold')
    ax2.set_ylim(0, 100)
    ax2.legend(loc='best', fontsize=9, framealpha=0.9)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Volatility comparison (PyNance calculated)
    ax3 = axes[1, 0]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'volatility_30d' in metrics['data'].columns:
            vol_data = metrics['data']['volatility_30d'].dropna() * 100
            if len(vol_data) > 0:
                ax3.plot(vol_data.index, vol_data.values, label=f"{ticker} (PyNance)", linewidth=1.5)
    ax3.set_ylabel('Volatility (%) - Annualized', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax3.set_title('30-Day Rolling Volatility (PyNance Calculation)', fontsize=13, fontweight='bold')
    ax3.legend(loc='best', fontsize=9, framealpha=0.9)
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Returns distribution (PyNance calculated)
    ax4 = axes[1, 1]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'daily_return' in metrics['data'].columns:
            returns = metrics['data']['daily_return'].dropna() * 100
            if len(returns) > 0:
                ax4.hist(returns, bins=50, alpha=0.6, label=f"{ticker} (PyNance)", density=True, edgecolor='black', linewidth=0.5)
    ax4.set_xlabel('Daily Return (%)', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Probability Density', fontsize=12, fontweight='bold')
    ax4.set_title('Daily Returns Distribution (PyNance Calculation)', fontsize=13, fontweight='bold')
    ax4.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Zero Return')
    ax4.legend(loc='best', fontsize=9, framealpha=0.9)
    ax4.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'stocks_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Comparison visualization created with PyNance metrics!")


## 8. PyNance Financial Metrics Visualization

Visualize the key financial metrics calculated using PyNance: volatility, beta, and returns.


In [ ]:
# Create visualization for PyNance financial metrics
if pynance_metrics:
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # Get date range
    sample_ticker = list(pynance_metrics.keys())[0]
    if 'data' in pynance_metrics[sample_ticker]:
        sample_df = pynance_metrics[sample_ticker]['data']
        date_range_str = f"{sample_df.index.min().strftime('%Y-%m-%d')} to {sample_df.index.max().strftime('%Y-%m-%d')}"
    else:
        date_range_str = "N/A"
    
    fig.suptitle(f'PyNance Financial Metrics Analysis\nTime Window: {date_range_str}', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    # 1. Cumulative Returns (PyNance calculated)
    ax1 = axes[0, 0]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'cumulative_return' in metrics['data'].columns:
            returns = metrics['data']['cumulative_return'].dropna() * 100
            if len(returns) > 0:
                ax1.plot(returns.index, returns.values, label=f"{ticker} (PyNance)", linewidth=2)
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax1.set_ylabel('Cumulative Return (%)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax1.set_title('Cumulative Returns Over Time (PyNance Calculation)', fontsize=13, fontweight='bold')
    ax1.legend(loc='best', fontsize=10, framealpha=0.9)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. Rolling Volatility (PyNance calculated)
    ax2 = axes[0, 1]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'volatility_30d' in metrics['data'].columns:
            vol_data = metrics['data']['volatility_30d'].dropna() * 100
            if len(vol_data) > 0:
                ax2.plot(vol_data.index, vol_data.values, label=f"{ticker} (PyNance)", linewidth=1.5)
    ax2.set_ylabel('30-Day Rolling Volatility (%) - Annualized', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax2.set_title('Rolling Volatility - Risk Measure (PyNance Calculation)', fontsize=13, fontweight='bold')
    ax2.legend(loc='best', fontsize=10, framealpha=0.9)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Beta comparison (if available)
    ax3 = axes[1, 0]
    beta_data = []
    tickers_list = []
    for ticker, metrics in pynance_metrics.items():
        if 'beta' in metrics and not pd.isna(metrics.get('beta')):
            beta_data.append(metrics['beta'])
            tickers_list.append(ticker)
    
    if beta_data:
        bars = ax3.bar(tickers_list, beta_data, alpha=0.7, color=['green' if b < 1 else 'orange' if b < 1.5 else 'red' for b in beta_data])
        ax3.axhline(y=1.0, color='black', linestyle='--', linewidth=2, label='Market Beta (1.0)')
        ax3.set_ylabel('Beta Coefficient', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Stock Ticker', fontsize=12, fontweight='bold')
        ax3.set_title('Beta - Market Sensitivity (PyNance Calculation)', fontsize=13, fontweight='bold')
        ax3.legend(loc='best', fontsize=10, framealpha=0.9)
        ax3.grid(True, alpha=0.3, linestyle='--', axis='y')
        
        # Add value labels on bars
        for i, (bar, beta) in enumerate(zip(bars, beta_data)):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{beta:.2f}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
    else:
        ax3.text(0.5, 0.5, 'Beta data not available\n(requires benchmark data)', 
                ha='center', va='center', transform=ax3.transAxes, fontsize=12)
        ax3.set_title('Beta - Market Sensitivity (PyNance Calculation)', fontsize=13, fontweight='bold')
    
    # 4. Drawdown analysis (PyNance calculated)
    ax4 = axes[1, 1]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'drawdown' in metrics['data'].columns:
            drawdown = metrics['data']['drawdown'].dropna() * 100
            if len(drawdown) > 0:
                ax4.fill_between(drawdown.index, drawdown.values, 0, 
                                alpha=0.5, label=f"{ticker} (PyNance)")
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax4.set_ylabel('Drawdown (%)', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax4.set_title('Drawdown Analysis - Risk Measure (PyNance Calculation)', fontsize=13, fontweight='bold')
    ax4.legend(loc='best', fontsize=10, framealpha=0.9)
    ax4.grid(True, alpha=0.3, linestyle='--')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'pynance_metrics_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ PyNance financial metrics visualization created!")


## 9. Summary and Key Insights


In [ ]:
print("=" * 80)
print("QUANTITATIVE ANALYSIS SUMMARY - NOVA FINANCIAL SOLUTIONS")
print("=" * 80)

print(f"\n📊 Stocks Analyzed: {len(indicators_data)}")
for ticker in indicators_data.keys():
    df = indicators_data[ticker]
    print(f"\n{'='*60}")
    print(f"📈 {ticker} Analysis")
    print(f"{'='*60}")
    print(f"  • Data points: {len(df):,}")
    print(f"  • Date range: {df.index.min().date()} to {df.index.max().date()}")
    print(f"  • Current price: ${df['close'].iloc[-1]:.2f}")
    print(f"  • Price change: {((df['close'].iloc[-1] / df['close'].iloc[0]) - 1) * 100:.2f}%")
    
    # RSI status with interpretation
    current_rsi = df['RSI'].iloc[-1]
    if current_rsi > 70:
        rsi_status = "Overbought ⚠️"
        rsi_decision = "Consider taking profits or avoiding new long positions"
    elif current_rsi < 30:
        rsi_status = "Oversold 📉"
        rsi_decision = "Potential buying opportunity, but monitor for momentum"
    else:
        rsi_status = "Neutral ✓"
        rsi_decision = "No extreme conditions detected"
    print(f"  • Current RSI: {current_rsi:.2f} ({rsi_status})")
    print(f"    → Trading Decision: {rsi_decision}")
    
    # MACD signal with interpretation
    current_macd = df['MACD'].iloc[-1]
    current_signal = df['MACD_signal'].iloc[-1]
    if current_macd > current_signal:
        macd_signal = "Bullish 📈"
        macd_decision = "Momentum is positive - consider long positions"
    else:
        macd_signal = "Bearish 📉"
        macd_decision = "Momentum is negative - consider short positions or exit"
    print(f"  • MACD signal: {macd_signal} (MACD: {current_macd:.2f}, Signal: {current_signal:.2f})")
    print(f"    → Trading Decision: {macd_decision}")
    
    # Moving average trend with interpretation
    current_price = df['close'].iloc[-1]
    sma_50 = df['SMA_50'].iloc[-1]
    sma_200 = df['SMA_200'].iloc[-1]
    if current_price > sma_50 > sma_200:
        trend = "Uptrend 📈"
        ma_decision = "Strong uptrend - favorable for long positions"
    elif current_price < sma_50 < sma_200:
        trend = "Downtrend 📉"
        ma_decision = "Strong downtrend - consider short positions or exit"
    else:
        trend = "Sideways ↔️"
        ma_decision = "No clear trend - wait for confirmation"
    print(f"  • Trend: {trend}")
    print(f"    → Trading Decision: {ma_decision}")
    print(f"    → Price vs MAs: Price ${current_price:.2f} | SMA50 ${sma_50:.2f} | SMA200 ${sma_200:.2f}")

# PyNance Financial Metrics
if pynance_metrics:
    print(f"\n{'='*60}")
    print(f"💰 PyNance Financial Metrics Summary")
    print(f"{'='*60}")
    for ticker, metrics in pynance_metrics.items():
        if 'total_return' in metrics:
            print(f"\n📊 {ticker} Financial Metrics (PyNance Calculated):")
            print(f"  • Total Return: {metrics['total_return']*100:.2f}%")
            
            if metrics.get('volatility') and not pd.isna(metrics['volatility']):
                vol_pct = metrics['volatility'] * 100
                print(f"  • Volatility (Annualized): {vol_pct:.2f}%")
                if vol_pct > 30:
                    print(f"    → Risk Assessment: High volatility - increased risk")
                elif vol_pct > 20:
                    print(f"    → Risk Assessment: Moderate volatility")
                else:
                    print(f"    → Risk Assessment: Low volatility - relatively stable")
            
            if 'beta' in metrics and not pd.isna(metrics.get('beta')):
                beta = metrics['beta']
                print(f"  • Beta: {beta:.2f}")
                if beta > 1.5:
                    print(f"    → Risk Assessment: Highly sensitive to market movements (aggressive)")
                elif beta > 1.0:
                    print(f"    → Risk Assessment: More volatile than market")
                elif beta > 0.5:
                    print(f"    → Risk Assessment: Less volatile than market")
                else:
                    print(f"    → Risk Assessment: Low sensitivity to market (defensive)")
            
            if metrics.get('sharpe_ratio') and not pd.isna(metrics['sharpe_ratio']):
                sharpe = metrics['sharpe_ratio']
                print(f"  • Sharpe Ratio: {sharpe:.2f}")
                if sharpe > 2:
                    print(f"    → Risk-Adjusted Return: Excellent")
                elif sharpe > 1:
                    print(f"    → Risk-Adjusted Return: Good")
                else:
                    print(f"    → Risk-Adjusted Return: Below average")
            
            if metrics.get('max_drawdown') and not pd.isna(metrics['max_drawdown']):
                dd_pct = metrics['max_drawdown'] * 100
                print(f"  • Max Drawdown: {dd_pct:.2f}%")
                if abs(dd_pct) > 30:
                    print(f"    → Risk Assessment: Significant downside risk")
                elif abs(dd_pct) > 20:
                    print(f"    → Risk Assessment: Moderate downside risk")
                else:
                    print(f"    → Risk Assessment: Controlled downside risk")

print("\n" + "=" * 80)
print("✅ Quantitative Analysis Complete!")
print("=" * 80)
print("\n💡 Key Takeaways for Nova Financial Solutions:")
print("  • Technical indicators (RSI, MACD, MAs) provide entry/exit signals")
print("  • PyNance metrics (volatility, beta, returns) quantify risk and performance")
print("  • Combined analysis enables data-driven trading and risk management decisions")
print("=" * 80)
